# Multi-Model Behavioral Validation Analysis - Study 4

This notebook analyzes the results from Study 4 moral reasoning and risk-taking behavioral validation across multiple LLM models.

## Prerequisites
- Run `study_4_multi_model_simulation.ipynb` first to generate simulation results
- Results should be saved in `study_4_results/` directory

## Analysis Overview
1. **Data Loading**: Load saved simulation results and participant data
2. **Behavioral Analysis**: Analyze moral reasoning and risk-taking patterns
3. **Personality-Behavior Correlations**: Test personality-behavior relationships
4. **Cross-Model Validation**: Assess consistency across different LLMs
5. **Scenario Analysis**: Examine responses across different scenarios
6. **Empirical Validation**: Compare with human behavioral data if available
7. **Comprehensive Reporting**: Generate detailed analysis report
8. **Export Results**: Save all analysis outputs for further research

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, chi2_contingency
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

print("Analysis environment setup complete")

## 1. Data Loading and Preparation

In [ ]:
# Load simulation results
results_dir = Path('study_4_results')
if not results_dir.exists():
    raise FileNotFoundError("Results directory not found. Please run study_4_multi_model_simulation.ipynb first.")

# Load moral reasoning results
moral_results = {}
moral_dir = results_dir / 'moral'
if moral_dir.exists():
    for json_file in moral_dir.glob('moral_*.json'):
        filename = json_file.stem
        parts = filename.split('_')
        model = parts[1]
        temp_part = parts[2]
        temp_value = parts[3]
        temperature = f"{temp_part.replace('temp', '')}.{temp_value}"
        
        key = f"{model}_temp{temperature}"
        
        with open(json_file, 'r') as f:
            moral_results[key] = json.load(f)
        
        print(f"Loaded moral {key}: {len(moral_results[key])} participants")

# Load risk-taking results
risk_results = {}
risk_dir = results_dir / 'risk'
if risk_dir.exists():
    for json_file in risk_dir.glob('risk_*.json'):
        filename = json_file.stem
        parts = filename.split('_')
        model = parts[1]
        temp_part = parts[2]
        temp_value = parts[3]
        temperature = f"{temp_part.replace('temp', '')}.{temp_value}"
        
        key = f"{model}_temp{temperature}"
        
        with open(json_file, 'r') as f:
            risk_results[key] = json.load(f)
        
        print(f"Loaded risk {key}: {len(risk_results[key])} participants")

print(f"\nTotal moral result sets: {len(moral_results)}")
print(f"Total risk result sets: {len(risk_results)}")

In [ ]:
# Load participant data
participant_data_path = Path('../../study_4/data/york_data_clean.csv')
if participant_data_path.exists():
    participant_data = pd.read_csv(participant_data_path)
    print(f"Loaded participant data: {participant_data.shape}")
    
    # Extract personality columns
    personality_columns = [col for col in participant_data.columns if 'bfi' in col.lower()]
    print(f"Available personality columns: {len(personality_columns)}")
    
    # Look for Big Five domain scores
    big_five_columns = []
    for domain in ['extraversion', 'agreeableness', 'conscientiousness', 'neuroticism', 'openness']:
        matching_cols = [col for col in participant_data.columns if domain.lower() in col.lower()]
        if matching_cols:
            big_five_columns.extend(matching_cols)
    
    print(f"Big Five domain columns found: {big_five_columns}")
    
    # Extract empirical behavioral measures if available
    behavioral_columns = []
    for behavior in ['moral', 'risk', 'decision', 'choice']:
        matching_cols = [col for col in participant_data.columns if behavior.lower() in col.lower()]
        behavioral_columns.extend(matching_cols)
    
    print(f"Behavioral columns found: {behavioral_columns[:10]}...")  # Show first 10
    
else:
    print("Participant data not found - some analyses will be limited")
    participant_data = None
    big_five_columns = []
    behavioral_columns = []

In [ ]:
# Load experiment summary if available
summary_path = results_dir / 'study4_experiment_summary.json'
if summary_path.exists():
    with open(summary_path, 'r') as f:
        experiment_summary = json.load(f)
    print(f"\nExperiment conducted: {experiment_summary.get('timestamp', 'Unknown')}")
    print(f"Models tested: {experiment_summary.get('models_tested', [])}")
    print(f"Total participants: {experiment_summary.get('total_participants', 'Unknown')}")
    print(f"Moral scenarios: {experiment_summary.get('moral_scenarios', 'Unknown')}")
    print(f"Risk scenarios: {experiment_summary.get('risk_scenarios', 'Unknown')}")
else:
    print("Experiment summary not found")
    experiment_summary = {}

## 2. Data Processing and Structure

In [ ]:
def process_behavioral_results(results_dict, scenario_type):
    """
    Process behavioral simulation results into structured DataFrames.
    
    Returns:
    - results_df: Long format DataFrame with all responses
    - scenario_stats: Summary statistics by scenario and model
    """
    
    # Initialize storage
    results_list = []
    
    # Process each model-temperature combination
    for model_temp, results in results_dict.items():
        if not isinstance(results, list):
            print(f"Skipping {model_temp}: {results}")
            continue
            
        model_name = model_temp.split('_temp')[0]
        temperature = model_temp.split('_temp')[1]
        
        # Extract responses for each participant
        for i, result in enumerate(results):
            if isinstance(result, dict) and 'error' not in result:
                # Process each scenario response
                for scenario_key, response in result.items():
                    if scenario_key.startswith(f'{scenario_type}_scenario_'):
                        scenario_num = scenario_key.split('_')[-1]
                        
                        response_dict = {
                            'participant_id': i,
                            'model': model_name,
                            'temperature': temperature,
                            'scenario_type': scenario_type,
                            'scenario_number': scenario_num,
                            'response': response
                        }
                        
                        # Extract choice/decision if structured
                        if isinstance(response, dict):
                            response_dict.update({
                                'choice': response.get('choice', response.get('decision', None)),
                                'reasoning': response.get('reasoning', response.get('explanation', None)),
                                'confidence': response.get('confidence', None),
                                'raw_response': str(response)
                            })
                        else:
                            # Handle string responses
                            response_dict.update({
                                'choice': None,
                                'reasoning': str(response),
                                'confidence': None,
                                'raw_response': str(response)
                            })
                        
                        results_list.append(response_dict)
    
    # Create DataFrame
    results_df = pd.DataFrame(results_list)
    
    # Calculate scenario statistics
    scenario_stats = []
    if not results_df.empty:
        stats = results_df.groupby(['model', 'temperature', 'scenario_number']).agg({
            'participant_id': 'count',
            'choice': 'nunique',
            'confidence': 'mean'
        }).rename(columns={
            'participant_id': 'n_responses',
            'choice': 'n_unique_choices'
        }).reset_index()
        
        scenario_stats = stats
    
    return results_df, scenario_stats

# Process both behavioral domains
print("Processing moral reasoning results...")
moral_df, moral_stats = process_behavioral_results(moral_results, 'moral')

print("Processing risk-taking results...")
risk_df, risk_stats = process_behavioral_results(risk_results, 'risk')

# Combine results
combined_behavioral_df = pd.concat([moral_df, risk_df], ignore_index=True)
combined_stats_df = pd.concat([moral_stats, risk_stats], ignore_index=True)

print(f"\nMoral reasoning results: {moral_df.shape}")
print(f"Risk-taking results: {risk_df.shape}")
print(f"Combined behavioral results: {combined_behavioral_df.shape}")
print(f"Combined statistics: {combined_stats_df.shape}")

if not combined_behavioral_df.empty:
    print(f"\nAvailable models: {combined_behavioral_df['model'].unique()}")
    print(f"Available temperatures: {combined_behavioral_df['temperature'].unique()}")
    print(f"Available scenario types: {combined_behavioral_df['scenario_type'].unique()}")
    print(f"Available scenarios: {sorted(combined_behavioral_df['scenario_number'].unique())}")

## 3. Behavioral Pattern Analysis

In [ ]:
def analyze_behavioral_patterns(combined_behavioral_df, combined_stats_df):
    """Analyze behavioral patterns across models and scenarios."""
    
    if combined_behavioral_df.empty:
        print("No behavioral data to analyze")
        return
    
    print("=== BEHAVIORAL PATTERN ANALYSIS ===")
    
    # 1. Response completion rates
    print("\n1. Response Completion Rates:")
    completion_rates = combined_behavioral_df.groupby(['scenario_type', 'model', 'temperature']).agg({
        'participant_id': 'count'
    }).rename(columns={'participant_id': 'n_responses'})
    print(completion_rates)
    
    # 2. Choice diversity by scenario type
    print("\n2. Choice Diversity by Scenario Type:")
    if 'choice' in combined_behavioral_df.columns:
        choice_diversity = combined_behavioral_df.groupby(['scenario_type', 'model']).agg({
            'choice': ['count', 'nunique']
        })
        choice_diversity.columns = ['total_responses', 'unique_choices']
        choice_diversity['diversity_ratio'] = choice_diversity['unique_choices'] / choice_diversity['total_responses']
        print(choice_diversity.round(3))
    
    # 3. Confidence patterns (if available)
    if 'confidence' in combined_behavioral_df.columns:
        print("\n3. Confidence Patterns:")
        confidence_stats = combined_behavioral_df.groupby(['scenario_type', 'model'])['confidence'].agg([
            'count', 'mean', 'std'
        ]).round(3)
        print(confidence_stats)
    
    # 4. Scenario-level analysis
    print("\n4. Scenario-Level Response Patterns:")
    if not combined_stats_df.empty:
        scenario_summary = combined_stats_df.groupby(['scenario_number', 'model']).agg({
            'n_responses': 'mean',
            'n_unique_choices': 'mean'
        }).round(2)
        print(scenario_summary.head(10))  # Show first 10 scenarios
    
    # 5. Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Response rates by model and scenario type
    if not combined_behavioral_df.empty:
        response_counts = combined_behavioral_df.groupby(['model', 'scenario_type']).size().unstack(fill_value=0)
        response_counts.plot(kind='bar', ax=axes[0, 0])
        axes[0, 0].set_title('Response Counts by Model and Scenario Type')
        axes[0, 0].set_ylabel('Number of Responses')
        axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Choice diversity by model
    if 'choice' in combined_behavioral_df.columns:
        moral_data = combined_behavioral_df[combined_behavioral_df['scenario_type'] == 'moral']
        risk_data = combined_behavioral_df[combined_behavioral_df['scenario_type'] == 'risk']
        
        if not moral_data.empty:
            moral_diversity = moral_data.groupby('model')['choice'].nunique()
            moral_diversity.plot(kind='bar', ax=axes[0, 1], color='skyblue', alpha=0.7, label='Moral')
        
        if not risk_data.empty:
            risk_diversity = risk_data.groupby('model')['choice'].nunique()
            risk_diversity.plot(kind='bar', ax=axes[0, 1], color='lightcoral', alpha=0.7, label='Risk')
        
        axes[0, 1].set_title('Choice Diversity by Model')
        axes[0, 1].set_ylabel('Number of Unique Choices')
        axes[0, 1].legend()
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Temperature effects
    if len(combined_behavioral_df['temperature'].unique()) > 1:
        temp_effects = combined_behavioral_df.groupby(['temperature', 'scenario_type']).size().unstack(fill_value=0)
        temp_effects.plot(kind='bar', ax=axes[1, 0])
        axes[1, 0].set_title('Temperature Effects on Response Patterns')
        axes[1, 0].set_ylabel('Number of Responses')
        axes[1, 0].tick_params(axis='x', rotation=0)
    
    # Scenario difficulty (response variability)
    if not combined_stats_df.empty and 'n_unique_choices' in combined_stats_df.columns:
        scenario_difficulty = combined_stats_df.groupby('scenario_number')['n_unique_choices'].mean().sort_values()
        scenario_difficulty.plot(kind='bar', ax=axes[1, 1])
        axes[1, 1].set_title('Scenario Difficulty (Choice Variability)')
        axes[1, 1].set_ylabel('Average Unique Choices')
        axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('study_4_results/behavioral_patterns.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return completion_rates, choice_diversity if 'choice' in combined_behavioral_df.columns else None

# Run behavioral pattern analysis
completion_rates, choice_diversity = analyze_behavioral_patterns(combined_behavioral_df, combined_stats_df)

## 4. Personality-Behavior Correlations

In [ ]:
def analyze_personality_behavior_correlations(combined_behavioral_df, participant_data, big_five_columns):
    """Analyze correlations between personality traits and behavioral responses."""
    
    if combined_behavioral_df.empty or participant_data is None or not big_five_columns:
        print("Insufficient data for personality-behavior correlation analysis")
        return None
    
    print("=== PERSONALITY-BEHAVIOR CORRELATION ANALYSIS ===")
    
    # Extract Big Five scores for participants
    n_participants = len(combined_behavioral_df['participant_id'].unique())
    personality_scores = participant_data[big_five_columns].iloc[:n_participants]
    
    print(f"Analyzing {len(big_five_columns)} personality dimensions for {n_participants} participants")
    
    correlation_results = {}
    
    # For each model and scenario type
    for model in combined_behavioral_df['model'].unique():
        for scenario_type in combined_behavioral_df['scenario_type'].unique():
            subset = combined_behavioral_df[
                (combined_behavioral_df['model'] == model) & 
                (combined_behavioral_df['scenario_type'] == scenario_type)
            ]
            
            if subset.empty:
                continue
            
            key = f"{model}_{scenario_type}"
            
            # Analyze different aspects of behavior
            behavior_metrics = {}
            
            # 1. Choice consistency (if choices are available)
            if 'choice' in subset.columns:
                # Calculate choice consistency per participant
                choice_consistency = subset.groupby('participant_id')['choice'].apply(
                    lambda x: len(x.unique()) / len(x) if len(x) > 0 else 0
                )
                behavior_metrics['choice_diversity'] = choice_consistency
            
            # 2. Response length (reasoning complexity)
            if 'reasoning' in subset.columns:
                reasoning_length = subset.groupby('participant_id')['reasoning'].apply(
                    lambda x: np.mean([len(str(reason)) for reason in x if pd.notna(reason)])
                )
                behavior_metrics['reasoning_complexity'] = reasoning_length
            
            # 3. Confidence levels (if available)
            if 'confidence' in subset.columns:
                avg_confidence = subset.groupby('participant_id')['confidence'].mean()
                behavior_metrics['avg_confidence'] = avg_confidence
            
            # Calculate correlations with personality traits
            trait_correlations = {}
            
            for trait in big_five_columns:
                trait_values = personality_scores[trait]
                
                for behavior_name, behavior_values in behavior_metrics.items():
                    # Align data
                    aligned_trait = []
                    aligned_behavior = []
                    
                    for participant_id in behavior_values.index:
                        if participant_id < len(trait_values):
                            trait_val = trait_values.iloc[participant_id]
                            behavior_val = behavior_values.iloc[participant_id]
                            
                            if pd.notna(trait_val) and pd.notna(behavior_val):
                                aligned_trait.append(trait_val)
                                aligned_behavior.append(behavior_val)
                    
                    # Calculate correlation
                    if len(aligned_trait) > 5:  # Minimum sample size
                        corr, p_value = pearsonr(aligned_trait, aligned_behavior)
                        if not np.isnan(corr):
                            trait_correlations[f"{trait}_{behavior_name}"] = {
                                'correlation': corr,
                                'p_value': p_value,
                                'n_participants': len(aligned_trait)
                            }
            
            if trait_correlations:
                correlation_results[key] = trait_correlations
    
    # Display results
    if correlation_results:
        print("\nSignificant Personality-Behavior Correlations (p < 0.05):")
        
        significant_results = []
        for model_scenario, correlations in correlation_results.items():
            for trait_behavior, stats in correlations.items():
                if stats['p_value'] < 0.05:
                    significant_results.append({
                        'model_scenario': model_scenario,
                        'trait_behavior': trait_behavior,
                        'correlation': stats['correlation'],
                        'p_value': stats['p_value'],
                        'n_participants': stats['n_participants']
                    })
        
        if significant_results:
            sig_df = pd.DataFrame(significant_results)
            sig_df = sig_df.sort_values('correlation', key=abs, ascending=False)
            print(sig_df.head(10).round(3))
            
            # Create correlation heatmap
            if len(sig_df) > 5:
                # Pivot for heatmap
                heatmap_data = sig_df.pivot_table(
                    index='model_scenario', 
                    columns='trait_behavior', 
                    values='correlation'
                )
                
                plt.figure(figsize=(14, 8))
                sns.heatmap(heatmap_data, annot=True, cmap='RdBu_r', center=0, 
                           fmt='.2f', cbar_kws={'label': 'Correlation'})
                plt.title('Significant Personality-Behavior Correlations')
                plt.ylabel('Model_ScenarioType')
                plt.xlabel('Trait_Behavior')
                plt.xticks(rotation=45, ha='right')
                plt.yticks(rotation=0)
                plt.tight_layout()
                plt.savefig('study_4_results/personality_behavior_correlations.png', dpi=300, bbox_inches='tight')
                plt.show()
        else:
            print("No significant correlations found")
    
    return correlation_results

# Run personality-behavior correlation analysis
personality_behavior_correlations = analyze_personality_behavior_correlations(
    combined_behavioral_df, participant_data, big_five_columns
)

## 5. Cross-Model Validation

In [ ]:
def analyze_cross_model_behavioral_consistency(combined_behavioral_df):
    """Assess consistency of behavioral responses across different LLMs."""
    
    if combined_behavioral_df.empty:
        print("No behavioral data for cross-model analysis")
        return
    
    print("=== CROSS-MODEL BEHAVIORAL CONSISTENCY ===")
    
    # 1. Response similarity across models
    print("\n1. Response Similarity Across Models:")
    
    model_similarities = {}
    
    for scenario_type in combined_behavioral_df['scenario_type'].unique():
        scenario_data = combined_behavioral_df[combined_behavioral_df['scenario_type'] == scenario_type]
        
        models = scenario_data['model'].unique()
        similarities = []
        model_pairs = []
        
        for i, model1 in enumerate(models):
            for j, model2 in enumerate(models):
                if i >= j:
                    continue
                
                # Get responses for same scenarios from both models
                model1_data = scenario_data[scenario_data['model'] == model1]
                model2_data = scenario_data[scenario_data['model'] == model2]
                
                if model1_data.empty or model2_data.empty:
                    continue
                
                # Calculate agreement for each scenario
                scenario_agreements = []
                
                for scenario_num in scenario_data['scenario_number'].unique():
                    model1_scenario = model1_data[model1_data['scenario_number'] == scenario_num]
                    model2_scenario = model2_data[model2_data['scenario_number'] == scenario_num]
                    
                    if not model1_scenario.empty and not model2_scenario.empty:
                        # Compare choices if available
                        if 'choice' in scenario_data.columns:
                            # Calculate choice agreement rate
                            common_participants = set(model1_scenario['participant_id']) & set(model2_scenario['participant_id'])
                            
                            if common_participants:
                                agreements = []
                                for participant in common_participants:
                                    choice1 = model1_scenario[model1_scenario['participant_id'] == participant]['choice'].iloc[0]
                                    choice2 = model2_scenario[model2_scenario['participant_id'] == participant]['choice'].iloc[0]
                                    
                                    if pd.notna(choice1) and pd.notna(choice2):
                                        agreements.append(choice1 == choice2)
                                
                                if agreements:
                                    agreement_rate = np.mean(agreements)
                                    scenario_agreements.append(agreement_rate)
                
                if scenario_agreements:
                    avg_agreement = np.mean(scenario_agreements)
                    similarities.append(avg_agreement)
                    model_pairs.append(f'{model1} vs {model2}')
        
        if similarities:
            model_similarities[scenario_type] = {
                'similarities': similarities,
                'pairs': model_pairs,
                'mean_similarity': np.mean(similarities),
                'std_similarity': np.std(similarities)
            }
            
            print(f"\n{scenario_type.upper()} Scenarios:")
            print(f"  Mean inter-model agreement: {np.mean(similarities):.3f}")
            print(f"  Std inter-model agreement: {np.std(similarities):.3f}")
            for pair, sim in zip(model_pairs, similarities):
                print(f"    {pair}: {sim:.3f}")
    
    # 2. Temperature consistency within models
    print("\n2. Temperature Consistency Within Models:")
    
    temp_consistency = {}
    
    for scenario_type in combined_behavioral_df['scenario_type'].unique():
        scenario_data = combined_behavioral_df[combined_behavioral_df['scenario_type'] == scenario_type]
        
        for model in scenario_data['model'].unique():
            temp0_data = scenario_data[(scenario_data['model'] == model) & 
                                     (scenario_data['temperature'] == '0.0')]
            temp1_data = scenario_data[(scenario_data['model'] == model) & 
                                     (scenario_data['temperature'] == '1.0')]
            
            if not temp0_data.empty and not temp1_data.empty:
                # Calculate consistency across scenarios
                scenario_consistencies = []
                
                for scenario_num in scenario_data['scenario_number'].unique():
                    temp0_scenario = temp0_data[temp0_data['scenario_number'] == scenario_num]
                    temp1_scenario = temp1_data[temp1_data['scenario_number'] == scenario_num]
                    
                    if not temp0_scenario.empty and not temp1_scenario.empty:
                        # Compare choices if available
                        if 'choice' in scenario_data.columns:
                            common_participants = set(temp0_scenario['participant_id']) & set(temp1_scenario['participant_id'])
                            
                            if common_participants:
                                agreements = []
                                for participant in common_participants:
                                    choice0 = temp0_scenario[temp0_scenario['participant_id'] == participant]['choice'].iloc[0]
                                    choice1 = temp1_scenario[temp1_scenario['participant_id'] == participant]['choice'].iloc[0]
                                    
                                    if pd.notna(choice0) and pd.notna(choice1):
                                        agreements.append(choice0 == choice1)
                                
                                if agreements:
                                    agreement_rate = np.mean(agreements)
                                    scenario_consistencies.append(agreement_rate)
                
                if scenario_consistencies:
                    temp_consistency[f"{scenario_type}_{model}"] = {
                        'mean_consistency': np.mean(scenario_consistencies),
                        'std_consistency': np.std(scenario_consistencies),
                        'n_scenarios': len(scenario_consistencies)
                    }
                    
                    print(f"  {scenario_type}_{model}: {np.mean(scenario_consistencies):.3f} "
                         f"(±{np.std(scenario_consistencies):.3f})")
    
    # 3. Visualization
    if model_similarities:
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Inter-model agreement
        scenario_types = list(model_similarities.keys())
        mean_sims = [model_similarities[s]['mean_similarity'] for s in scenario_types]
        std_sims = [model_similarities[s]['std_similarity'] for s in scenario_types]
        
        axes[0].bar(scenario_types, mean_sims, yerr=std_sims, capsize=5)
        axes[0].set_title('Inter-Model Agreement by Scenario Type')
        axes[0].set_ylabel('Mean Agreement Rate')
        axes[0].set_ylim(0, 1)
        
        # Temperature consistency
        if temp_consistency:
            models = list(temp_consistency.keys())
            consistencies = [temp_consistency[m]['mean_consistency'] for m in models]
            
            axes[1].bar(range(len(models)), consistencies)
            axes[1].set_title('Temperature Consistency')
            axes[1].set_ylabel('Mean Agreement Rate')
            axes[1].set_xticks(range(len(models)))
            axes[1].set_xticklabels(models, rotation=45, ha='right')
            axes[1].set_ylim(0, 1)
        
        plt.tight_layout()
        plt.savefig('study_4_results/cross_model_behavioral_consistency.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    return model_similarities, temp_consistency

# Run cross-model behavioral consistency analysis
model_similarities, temp_consistency = analyze_cross_model_behavioral_consistency(combined_behavioral_df)

## 6. Scenario Analysis

In [ ]:
def analyze_scenario_difficulty(combined_behavioral_df, combined_stats_df):
    """Analyze which scenarios are most challenging or show most variability."""
    
    if combined_behavioral_df.empty:
        print("No data for scenario analysis")
        return
    
    print("=== SCENARIO DIFFICULTY ANALYSIS ===")
    
    # 1. Response variability by scenario
    scenario_metrics = {}
    
    for scenario_type in combined_behavioral_df['scenario_type'].unique():
        scenario_data = combined_behavioral_df[combined_behavioral_df['scenario_type'] == scenario_type]
        
        scenario_analysis = {}
        
        for scenario_num in scenario_data['scenario_number'].unique():
            scenario_subset = scenario_data[scenario_data['scenario_number'] == scenario_num]
            
            if scenario_subset.empty:
                continue
            
            metrics = {
                'n_responses': len(scenario_subset),
                'n_models': len(scenario_subset['model'].unique()),
                'response_rate': len(scenario_subset) / (len(scenario_subset['model'].unique()) * len(scenario_subset['participant_id'].unique()))
            }
            
            # Choice diversity
            if 'choice' in scenario_subset.columns:
                unique_choices = scenario_subset['choice'].nunique()
                total_responses = len(scenario_subset['choice'].dropna())
                metrics['choice_diversity'] = unique_choices / total_responses if total_responses > 0 else 0
                metrics['n_unique_choices'] = unique_choices
            
            # Response length variability
            if 'reasoning' in scenario_subset.columns:
                response_lengths = [len(str(r)) for r in scenario_subset['reasoning'] if pd.notna(r)]
                if response_lengths:
                    metrics['avg_response_length'] = np.mean(response_lengths)
                    metrics['response_length_std'] = np.std(response_lengths)
            
            # Confidence variability
            if 'confidence' in scenario_subset.columns:
                confidence_values = scenario_subset['confidence'].dropna()
                if len(confidence_values) > 0:
                    metrics['avg_confidence'] = confidence_values.mean()
                    metrics['confidence_std'] = confidence_values.std()
            
            scenario_analysis[scenario_num] = metrics
        
        scenario_metrics[scenario_type] = scenario_analysis
    
    # 2. Identify most and least challenging scenarios
    print("\n2. Scenario Difficulty Rankings:")
    
    for scenario_type, scenarios in scenario_metrics.items():
        print(f"\n{scenario_type.upper()} Scenarios:")
        
        # Sort by choice diversity (higher = more challenging/ambiguous)
        if scenarios and 'choice_diversity' in next(iter(scenarios.values())):
            diversity_ranking = sorted(scenarios.items(), 
                                     key=lambda x: x[1]['choice_diversity'], reverse=True)
            
            print("  Most Challenging (High Choice Diversity):")
            for scenario_num, metrics in diversity_ranking[:3]:
                print(f"    Scenario {scenario_num}: {metrics['choice_diversity']:.3f} diversity, "
                     f"{metrics['n_unique_choices']} unique choices")
            
            print("  Least Challenging (Low Choice Diversity):")
            for scenario_num, metrics in diversity_ranking[-3:]:
                print(f"    Scenario {scenario_num}: {metrics['choice_diversity']:.3f} diversity, "
                     f"{metrics['n_unique_choices']} unique choices")
    
    # 3. Visualization
    if scenario_metrics:
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        for i, (scenario_type, scenarios) in enumerate(scenario_metrics.items()):
            if i >= 2:  # Only plot first 2 scenario types
                break
            
            scenario_nums = list(scenarios.keys())
            
            # Choice diversity
            if scenarios and 'choice_diversity' in next(iter(scenarios.values())):
                diversities = [scenarios[s]['choice_diversity'] for s in scenario_nums]
                axes[i, 0].bar(scenario_nums, diversities)
                axes[i, 0].set_title(f'{scenario_type.title()} - Choice Diversity')
                axes[i, 0].set_ylabel('Choice Diversity Ratio')
                axes[i, 0].set_xlabel('Scenario Number')
            
            # Response length
            if scenarios and 'avg_response_length' in next(iter(scenarios.values())):
                lengths = [scenarios[s].get('avg_response_length', 0) for s in scenario_nums]
                axes[i, 1].bar(scenario_nums, lengths)
                axes[i, 1].set_title(f'{scenario_type.title()} - Response Length')
                axes[i, 1].set_ylabel('Average Response Length')
                axes[i, 1].set_xlabel('Scenario Number')
        
        plt.tight_layout()
        plt.savefig('study_4_results/scenario_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    return scenario_metrics

# Run scenario difficulty analysis
scenario_difficulty = analyze_scenario_difficulty(combined_behavioral_df, combined_stats_df)

## 7. Comprehensive Report Generation

In [ ]:
def generate_study4_comprehensive_report(
    combined_behavioral_df, combined_stats_df, completion_rates, choice_diversity,
    personality_behavior_correlations, model_similarities, temp_consistency,
    scenario_difficulty, experiment_summary
):
    """Generate a comprehensive analysis report for Study 4."""
    
    report = []
    report.append("# Study 4: Multi-Model Behavioral Validation Analysis Report")
    report.append(f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report.append("\n" + "="*80)
    
    # Executive Summary
    report.append("\n## Executive Summary")
    if experiment_summary:
        report.append(f"- **Study Focus**: Behavioral validation through moral reasoning and risk-taking scenarios")
        report.append(f"- **Participants Analyzed**: {experiment_summary.get('total_participants', 'Unknown')}")
        report.append(f"- **Models Tested**: {experiment_summary.get('models_tested', [])}")
        report.append(f"- **Behavioral Domains**: Moral reasoning and risk-taking decision making")
        report.append(f"- **Temperature Settings**: {experiment_summary.get('temperatures', [])}")
    
    if not combined_behavioral_df.empty:
        total_responses = len(combined_behavioral_df)
        unique_scenarios = len(combined_behavioral_df['scenario_number'].unique())
        report.append(f"- **Total Behavioral Responses**: {total_responses}")
        report.append(f"- **Unique Scenarios Tested**: {unique_scenarios}")
    
    # Behavioral Pattern Analysis
    report.append("\n## Behavioral Pattern Analysis")
    
    if completion_rates is not None:
        report.append("\n### Response Completion:")
        # Summarize completion rates
        avg_completion = completion_rates.mean().iloc[0] if not completion_rates.empty else 0
        report.append(f"- **Average Response Rate**: {avg_completion:.1f} responses per model-scenario combination")
    
    if choice_diversity is not None and not choice_diversity.empty:
        report.append("\n### Choice Diversity:")
        avg_diversity = choice_diversity['diversity_ratio'].mean()
        report.append(f"- **Average Choice Diversity**: {avg_diversity:.3f} (ratio of unique choices to total responses)")
        
        # Find most and least diverse models
        most_diverse = choice_diversity['diversity_ratio'].idxmax()[1]  # Get model name
        least_diverse = choice_diversity['diversity_ratio'].idxmin()[1]
        report.append(f"- **Most Diverse Model**: {most_diverse}")
        report.append(f"- **Most Consistent Model**: {least_diverse}")
    
    # Personality-Behavior Relationships
    if personality_behavior_correlations:
        report.append("\n## Personality-Behavior Relationships")
        
        # Count significant correlations
        total_correlations = 0
        significant_correlations = 0
        
        for model_scenario, correlations in personality_behavior_correlations.items():
            for trait_behavior, stats in correlations.items():
                total_correlations += 1
                if stats['p_value'] < 0.05:
                    significant_correlations += 1
        
        if total_correlations > 0:
            sig_rate = (significant_correlations / total_correlations) * 100
            report.append(f"\n- **Significant Correlations Found**: {significant_correlations}/{total_correlations} ({sig_rate:.1f}%)")
            
            # Find strongest correlations
            strongest_corrs = []
            for model_scenario, correlations in personality_behavior_correlations.items():
                for trait_behavior, stats in correlations.items():
                    if stats['p_value'] < 0.05:
                        strongest_corrs.append((trait_behavior, stats['correlation'], model_scenario))
            
            if strongest_corrs:
                strongest_corrs.sort(key=lambda x: abs(x[1]), reverse=True)
                report.append("\n### Strongest Personality-Behavior Relationships:")
                for trait_behavior, corr, model_scenario in strongest_corrs[:5]:
                    report.append(f"- **{trait_behavior}** in {model_scenario}: r = {corr:.3f}")
    
    # Cross-Model Consistency
    report.append("\n## Cross-Model Consistency")
    
    if model_similarities:
        report.append("\n### Inter-Model Agreement:")
        for scenario_type, stats in model_similarities.items():
            mean_agreement = stats['mean_similarity']
            std_agreement = stats['std_similarity']
            report.append(f"- **{scenario_type.title()} Scenarios**: {mean_agreement:.3f} (±{std_agreement:.3f}) agreement rate")
    
    if temp_consistency:
        report.append("\n### Temperature Consistency:")
        temp_means = [stats['mean_consistency'] for stats in temp_consistency.values()]
        if temp_means:
            overall_temp_consistency = np.mean(temp_means)
            report.append(f"- **Overall Temperature Consistency**: {overall_temp_consistency:.3f}")
            
            # Most and least consistent models
            best_temp_model = max(temp_consistency.keys(), key=lambda x: temp_consistency[x]['mean_consistency'])
            worst_temp_model = min(temp_consistency.keys(), key=lambda x: temp_consistency[x]['mean_consistency'])
            report.append(f"- **Most Temperature-Consistent**: {best_temp_model}")
            report.append(f"- **Least Temperature-Consistent**: {worst_temp_model}")
    
    # Scenario Analysis
    if scenario_difficulty:
        report.append("\n## Scenario Analysis")
        
        for scenario_type, scenarios in scenario_difficulty.items():
            if scenarios:
                report.append(f"\n### {scenario_type.title()} Scenarios:")
                
                # Find most challenging scenarios
                if 'choice_diversity' in next(iter(scenarios.values())):
                    diversity_ranking = sorted(scenarios.items(), 
                                             key=lambda x: x[1]['choice_diversity'], reverse=True)
                    
                    most_challenging = diversity_ranking[0]
                    least_challenging = diversity_ranking[-1]
                    
                    report.append(f"- **Most Challenging**: Scenario {most_challenging[0]} "
                                f"({most_challenging[1]['choice_diversity']:.3f} diversity)")
                    report.append(f"- **Least Challenging**: Scenario {least_challenging[0]} "
                                f"({least_challenging[1]['choice_diversity']:.3f} diversity)")
    
    # Key Findings
    report.append("\n## Key Findings")
    
    findings = []
    
    # Model performance
    if model_similarities:
        all_agreements = [stats['mean_similarity'] for stats in model_similarities.values()]
        if all_agreements:
            overall_agreement = np.mean(all_agreements)
            findings.append(f"**Model Consistency**: Average inter-model agreement of {overall_agreement:.3f}")
    
    # Behavioral patterns
    if choice_diversity is not None and not choice_diversity.empty:
        avg_diversity = choice_diversity['diversity_ratio'].mean()
        findings.append(f"**Behavioral Diversity**: Average choice diversity of {avg_diversity:.3f} across all scenarios")
    
    # Personality relationships
    if personality_behavior_correlations:
        sig_count = sum(1 for correlations in personality_behavior_correlations.values() 
                       for stats in correlations.values() if stats['p_value'] < 0.05)
        if sig_count > 0:
            findings.append(f"**Personality Effects**: {sig_count} significant personality-behavior correlations identified")
    
    for finding in findings:
        report.append(f"\n- {finding}")
    
    # Recommendations
    report.append("\n## Recommendations")
    
    recommendations = [
        "**Behavioral Assessment**: Use scenarios with moderate choice diversity for reliable personality assessment",
        "**Model Selection**: Consider ensemble approaches for more robust behavioral predictions",
        "**Temperature Settings**: Use temperature = 0.0 for consistent behavioral assessment applications",
        "**Scenario Design**: Focus on scenarios that show consistent personality-behavior relationships",
        "**Future Research**: Investigate cultural and contextual factors in moral and risk-taking behaviors"
    ]
    
    for rec in recommendations:
        report.append(f"\n- {rec}")
    
    # Technical Notes
    report.append("\n## Technical Notes")
    technical_notes = [
        "Behavioral analysis based on moral reasoning and risk-taking scenarios",
        "Agreement rates calculated using choice overlap between models",
        "Correlations computed using Pearson's correlation coefficient",
        "Statistical significance testing performed where applicable",
        "Choice diversity calculated as ratio of unique choices to total responses"
    ]
    
    for note in technical_notes:
        report.append(f"- {note}")
    
    return "\n".join(report)

# Generate comprehensive report
study4_report = generate_study4_comprehensive_report(
    combined_behavioral_df, combined_stats_df, completion_rates, choice_diversity,
    personality_behavior_correlations, model_similarities, temp_consistency,
    scenario_difficulty, experiment_summary
)

print(study4_report)

# Save report to file
with open('study_4_results/comprehensive_analysis_report.md', 'w') as f:
    f.write(study4_report)

print("\n" + "="*80)
print("STUDY 4 ANALYSIS COMPLETE")
print("Report saved to: study_4_results/comprehensive_analysis_report.md")
print("="*80)

## 8. Export Results for Further Analysis

In [ ]:
def export_study4_results(
    combined_behavioral_df, combined_stats_df, completion_rates, choice_diversity,
    personality_behavior_correlations, scenario_difficulty
):
    """Export all Study 4 analysis results in formats suitable for further research."""
    
    output_dir = Path('study_4_results')
    output_dir.mkdir(exist_ok=True)
    
    print("Exporting Study 4 analysis results...")
    
    # 1. Export main behavioral results
    if not combined_behavioral_df.empty:
        combined_behavioral_df.to_csv(output_dir / 'behavioral_responses_combined.csv', index=False)
        print(f"✓ Combined behavioral responses: {len(combined_behavioral_df)} rows")
        
        # Separate by scenario type
        for scenario_type in combined_behavioral_df['scenario_type'].unique():
            scenario_data = combined_behavioral_df[combined_behavioral_df['scenario_type'] == scenario_type]
            scenario_data.to_csv(output_dir / f'behavioral_responses_{scenario_type}.csv', index=False)
            print(f"✓ {scenario_type.title()} responses: {len(scenario_data)} rows")
    
    if not combined_stats_df.empty:
        combined_stats_df.to_csv(output_dir / 'scenario_statistics.csv', index=False)
        print(f"✓ Scenario statistics: {len(combined_stats_df)} rows")
    
    # 2. Export completion rates
    if completion_rates is not None and not completion_rates.empty:
        completion_rates.to_csv(output_dir / 'response_completion_rates.csv')
        print(f"✓ Completion rates: {completion_rates.shape}")
    
    # 3. Export choice diversity metrics
    if choice_diversity is not None and not choice_diversity.empty:
        choice_diversity.to_csv(output_dir / 'choice_diversity_metrics.csv')
        print(f"✓ Choice diversity metrics: {choice_diversity.shape}")
    
    # 4. Export personality-behavior correlations
    if personality_behavior_correlations:
        correlation_flat = []
        for model_scenario, correlations in personality_behavior_correlations.items():
            for trait_behavior, stats in correlations.items():
                parts = model_scenario.split('_')
                model = parts[0]
                scenario_type = parts[1]
                
                trait_parts = trait_behavior.split('_')
                trait = '_'.join(trait_parts[:-1])
                behavior = trait_parts[-1]
                
                correlation_flat.append({
                    'model': model,
                    'scenario_type': scenario_type,
                    'personality_trait': trait,
                    'behavioral_measure': behavior,
                    'correlation': stats['correlation'],
                    'p_value': stats['p_value'],
                    'n_participants': stats['n_participants'],
                    'significant': stats['p_value'] < 0.05
                })
        
        correlation_df = pd.DataFrame(correlation_flat)
        correlation_df.to_csv(output_dir / 'personality_behavior_correlations.csv', index=False)
        print(f"✓ Personality-behavior correlations: {len(correlation_df)} rows")
    
    # 5. Export scenario difficulty analysis
    if scenario_difficulty:
        scenario_flat = []
        for scenario_type, scenarios in scenario_difficulty.items():
            for scenario_num, metrics in scenarios.items():
                record = {
                    'scenario_type': scenario_type,
                    'scenario_number': scenario_num
                }
                record.update(metrics)
                scenario_flat.append(record)
        
        scenario_df = pd.DataFrame(scenario_flat)
        scenario_df.to_csv(output_dir / 'scenario_difficulty_metrics.csv', index=False)
        print(f"✓ Scenario difficulty metrics: {len(scenario_df)} rows")
    
    # 6. Create analysis summary
    summary_stats = {
        'analysis_timestamp': pd.Timestamp.now().isoformat(),
        'study': 'Study 4 - Behavioral Validation',
        'n_participants': len(combined_behavioral_df['participant_id'].unique()) if not combined_behavioral_df.empty else 0,
        'n_models': len(combined_behavioral_df['model'].unique()) if not combined_behavioral_df.empty else 0,
        'n_temperatures': len(combined_behavioral_df['temperature'].unique()) if not combined_behavioral_df.empty else 0,
        'n_scenario_types': len(combined_behavioral_df['scenario_type'].unique()) if not combined_behavioral_df.empty else 0,
        'n_total_scenarios': len(combined_behavioral_df['scenario_number'].unique()) if not combined_behavioral_df.empty else 0,
        'n_total_responses': len(combined_behavioral_df) if not combined_behavioral_df.empty else 0
    }
    
    with open(output_dir / 'analysis_metadata.json', 'w') as f:
        json.dump(summary_stats, f, indent=2)
    print(f"✓ Analysis metadata saved")
    
    # 7. Create R-ready format
    if not combined_behavioral_df.empty:
        # Long format for R analysis
        r_format = combined_behavioral_df.copy()
        r_format.to_csv(output_dir / 'behavioral_responses_r_format.csv', index=False)
        print(f"✓ R-ready format: {len(r_format)} rows")
    
    print(f"\n📁 All Study 4 results exported to: {output_dir}/")
    print("Files available for further analysis:")
    for file in sorted(output_dir.glob('*.csv')):
        print(f"  - {file.name}")
    for file in sorted(output_dir.glob('*.json')):
        print(f"  - {file.name}")
    for file in sorted(output_dir.glob('*.png')):
        print(f"  - {file.name}")

# Export all results
export_study4_results(
    combined_behavioral_df, combined_stats_df, completion_rates, choice_diversity,
    personality_behavior_correlations, scenario_difficulty
)

print("\n🎉 STUDY 4 MULTI-MODEL ANALYSIS COMPLETE! 🎉")
print("\nNext steps:")
print("1. Review the comprehensive report: study_4_results/comprehensive_analysis_report.md")
print("2. Examine visualizations in study_4_results/")
print("3. Use exported CSV files for statistical analysis in R or Python")
print("4. Compare with Studies 2 and 3 using similar analysis frameworks")
print("5. Conduct advanced behavioral modeling and personality-behavior relationship analysis")